In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

Let's design an example with three agents using the CrewAI framework. We'll set up a scenario where these agents collaborate to produce an insightful article on a tech topic:

1. **Researcher Agent**: This agent's role is to conduct in-depth research on the specified tech topic, gathering key insights and latest developments.
2. **Analyst Agent**: This agent analyzes the information collected by the Researcher, identifying trends, opportunities, and challenges.
3. **Writer Agent**: After receiving the analyzed data, this agent will craft a well-written, engaging article on the topic.

In this setup:
- The **Researcher** gathers information using the `SerperDevTool` for searching relevant data.
- The **Analyst** processes this information to highlight important trends and insights.
- The **Writer** uses these insights to craft an article aimed at a general audience.

This example demonstrates how you can configure multiple agents with different roles and collaborative tasks to achieve a complex objective, leveraging the capabilities of the CrewAI framework.

### Creating an agent with all attributes

```python
agent = Agent(
  role='Data Analyst',
  goal='Extract actionable insights',
  backstory="""You're a data analyst at a large company.
  You're responsible for analyzing data and providing insights
  to the business.
  You're currently working on a project to analyze the
  performance of our marketing campaigns.""",
  tools=[my_tool1, my_tool2],  # Optional, defaults to an empty list
  llm=my_llm,  # Optional
  function_calling_llm=my_llm,  # Optional
  max_iter=15,  # Optional
  max_rpm=None, # Optional
  verbose=True,  # Optional
  allow_delegation=True,  # Optional
  step_callback=my_intermediate_step_callback,  # Optional
  cache=True  # Optional
)


In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [3]:
search_tool = SerperDevTool()

In [4]:
# os.environ["OPENAI_MODEL_NAME"]="gpt-4-0125-preview"
# os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo"
os.environ["OPENAI_MODEL_NAME"]="gpt-4-turbo-preview"

# My Agents

In [6]:
# General Fighter Stat Analysis Agent
agent1 = Agent(
    role="Fighter Stats Analyst",
    goal="""Analyze fighter's performance statistics and evaluate strengths and
    weaknesses based on fighter data""",
#     goal='Collect and analyze detailed data on fighters and fights',
    backstory=
        """As a seasoned analyst with a deep understanding of MMA/UFC, you specialize
        in breaking down fights and understanding fighter tendencies and histories.""",
    verbose=True,
    allow_delegation=True,
    tools=[search_tool],
)

# Recent Performance and Momentum Analysis Agent
agent2 = Agent(
    role="Historical Fight Analyst",
    goal="""Analyze previous fights to learn performance against similar opponents 
    as well to assess current momentum of the fighter""",
    backstory=
        """As a seasoned analyst with a deep understanding of MMA/UFC, you specialize
        in breaking down fights.
        Your expertise lies in dissecting historical fight outcomes and statistics,
        providing insights into how a fighter's recent experiences might influence their
        next performance.""",
    verbose=True,
    allow_delegation=True,
    tools=[search_tool],
)

# Predictor Agent
agent3 = Agent(
    role="Prediction Agent",
    goal="Predict outcomes of upcoming fights based on statistical analysis",
    backstory=
        """As a expert analyst and oddsmaker with a deep understanding of MMA/UFC,
        you specialize in breaking down and handicapping fights.
        Equipped with advanced predictive modeling and professional handicapping knowledge,
        you use and evaluate data to predict hypothetical fight outcomes with high accuracy.""",
    verbose=True,
    allow_delegation=True,
)

# # Psychological Analysis Agent
# agent999 = Agent(
#     role="Psychological Analysis",
#     goal="""Analyze fighters' psychological readiness and gauge mental state and 
#     confidence levels from pre-fight interviews and public appearances"""
#     verbose=True,
#     allow_delegation=True,
# )

In [9]:
# Run Tasks

fighter1 = input("Fighter 1: ")
fighter2 = input("Fighter 2: ")

task1 = Task(
    description=f"Research detailed statistics and historical performance of {fighter1}",
    expected_output=f"Comprehensive statistical report on {fighter1}.",
    agent=agent1,
    output_file='./logs/task1_output.txt'
)
task2 = Task(
    description=f"Research detailed statistics and historical performance of {fighter2}",
    expected_output=f"Comprehensive statistical report on {fighter2}.",
    agent=agent1,
    output_file='./logs/task2_output.txt'
)
task3 = Task(
    description=f"Analyze historical fight data and momentum of {fighter1}",
    expected_output=f"Detailed analysis of fight history and current momentum of {fighter1}.",
    agent=agent2,
    output_file='./logs/task3_output.txt'
)
task4 = Task(
    description=f"Analyze historical fight data and momentum of {fighter2}",
    expected_output=f"Detailed analysis of fight history and current momentum of {fighter2}.",
    agent=agent2,
    output_file='./logs/task4_output.txt'
)

final_task = Task(
    description=f"""Using the insights provided, predict the outcome of a hypothetical
    fight between {fighter1} and {fighter2}.""",
    expected_output=
    """Your prediction should be informative and insightful based on findings from your 
    research. You must include the round and method of victory in your prediciton.""",
    agent=agent3,
    output_file='./logs/final_task_output.txt'
)


Fighter 1: Michael Chandler
Fighter 2: Conor McGregor


In [10]:
# Assemble crew
crew = Crew(
    agents=[agent1, agent2, agent3],
    tasks=[task1, task2, task3, task4, final_task],
    verbose=2
#     verbose=True
#     process=Process.sequential,
#     full_output=True,
)

# Execute tasks
# crew.kickoff()
result = crew.kickoff() 
print(result)
# print(crew.usage_metrics)

# Write the output to the dynamically named file
with open(f"./fight-predictions/{fighter1}_v_{fighter2}.txt", "a") as file:
    file.write(str(result) + "\n")
    file.write("----------------------------------------\n")

 [DEBUG]: == Working Agent: Fighter Stats Analyst
 [INFO]: == Starting Task: Research detailed statistics and historical performance of Michael Chandler


> Entering new CrewAgentExecutor chain...
I need to gather detailed statistics and historical performance data on Michael Chandler to provide a comprehensive report.

Action: Search the internet
Action Input: {"search_query": "Michael Chandler detailed fight statistics and historical performance"} 


Search results: Title: Michael Chandler Record: 23-8-0 - Stats | UFC
Link: http://ufcstats.com/fighter-details/4b93a88f3b1de35b
Snippet: Michael Chandler Record: 23-8-0 ... Career statistics: SLpM: 4.89; Str. Acc.: 46%; SApM: 4.58; Str. Def: 46%.
---
Title: Michael Chandler (Lightweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/2504988/michael-chandler
Snippet: View the profile of the MMA fighter Michael Chandler from USA on ESPN. Get the latest news, live stats and MMA fight highlights.
---
Title: Michael Chandler 

Thought: 
I have gathered detailed statistics and historical performance information on Conor McGregor by searching the internet. Now, I need to compile all the relevant data into a comprehensive statistical report.

Final Answer: 
Conor McGregor's MMA Record:
- Record: 22-6-0
- UFC Record: 10-4-0
- Win Finish Percentage: 90.9%
- Quality Performance Percentage: 71.4%

Career Statistics:
- SLpM (Significant Strikes Landed per Minute): 5.32
- Str. Acc. (Significant Strike Accuracy): 49%
- SApM (Significant Strikes Absorbed per Minute): 4.66
- Str. Def (Significant Strike Defense): 54%

Other Information:
- Height: 5'9"
- Weight: 155 lbs
- Reach: 74"
- Weight Class: Lightweight
- Stance: Southpaw

McGregor is known for being the biggest pay-per-view draw in MMA history and has headlined the five highest-selling UFC PPV events. His fighting style includes a high win finish percentage and strong performance statistics, showcasing his striking abilities and overall skill in the Octagon.

> F

Final Answer: 
Conor McGregor, a renowned Irish professional mixed martial artist, has a professional MMA record of 22 wins and 6 losses. His UFC career statistics include a SLpM (Significant Strikes Landed per Minute) of 5.32, Str. Acc. (Significant Strike Accuracy) of 49%, SApM (Significant Strikes Absorbed per Minute) of 4.66, and Str. Def (Significant Strike Defense) of 54%.

McGregor's recent performances have been a mix of wins and losses, with his last fight being a loss to Dustin Poirier at UFC 264 on July 10, 2021. Despite the recent setbacks, McGregor remains a prominent figure in the UFC lightweight division with a high level of skill in striking and precision.

His fighting style is characterized by his powerful left hand and precision striking, often leading to quick finishes in his fights. McGregor's ability to control the distance and pace of the fight, combined with his mental warfare tactics, has made him a challenging opponent for many in the division.

In terms of mo

Thought:
I now know the final answer.

Final Answer:
Based on the analysis of Conor McGregor and Michael Chandler's fighting styles, recent performances, and statistical data, I predict that in a hypothetical fight between the two, Conor McGregor would emerge victorious in the second round by TKO (Technical Knockout). McGregor's precision striking, ability to control distance, and powerful left hand give him an edge in this matchup. While Michael Chandler is an aggressive fighter with a wrestling background, McGregor's skill in striking and past achievements suggest he would secure the win in the second round by overwhelming Chandler with his precision and striking prowess.

> Finished chain.
 [DEBUG]: == [Prediction Agent] Task output: Based on the analysis of Conor McGregor and Michael Chandler's fighting styles, recent performances, and statistical data, I predict that in a hypothetical fight between the two, Conor McGregor would emerge victorious in the second round by TKO (Technic

In [ ]:
# # Ex backstories
#     backstory=(
#         "With a strong analytical mind, you excel at interpreting data and extracting meaningful insights from complex information."
#     ),
#     tools=[search_tool]
    
#     backstory=(
#         "As a tech enthusiast and skilled researcher, you have a keen eye for detail and a deep understanding of technology trends."
#     ),
#     tools=[search_tool]
# )

In [ ]:
#
# # Task for the stats analyst
# stats_analysis_task = Task(
#     description="Perform detailed analysis of fighter statistics including win rates, knockout rates, and fighting style efficiencies.",
#     expected_output="A comprehensive report of fighter stats.",
#     agent=stats_analyst
# )

# # Task for the historical fight analyst
# historical_analysis_task = Task(
#     description="Review historical fight data to identify trends and performance against types of opponents.",
#     expected_output="Report on historical performance trends and fighter momentum.",
#     agent=fight_analyst
# )

# # Task for the prediction modeler
# prediction_task = Task(
#     description="Use statistical and historical data to predict outcomes of the next event.",
#     expected_output="Predictions for upcoming fight outcomes with probabilities.",
#     agent=prediction_modeler
# )

In [ ]:
# Ex Tasks

# # Define tasks for each fighter with separate agents
# task1 = Task(
#     description=f"Research detailed statistics and historical performance of {fighter1}",
#     agent=stats_analyst  # Assuming stats_analyst is already defined
# )

# task2 = Task(
#     description=f"Research detailed statistics and historical performance of {fighter2}",
#     agent=stats_analyst
# )

# task3 = Task(
#     description=f"Analyze historical fight data and momentum of {fighter1}",
#     agent=fight_analyst  # Assuming fight_analyst is already defined
# )

# task4 = Task(
#     description=f"Analyze historical fight data and momentum of {fighter2}",
#     agent=fight_analyst
# )

# # Assuming Crew and Process are already defined and imported
# crew = Crew(
#     agents=[stats_analyst, fight_analyst],
#     tasks=[task1, task2, task3, task4],
#     process=Process.sequential  # or Process.parallel if supported
# )

# # Kick off the crew's tasks
# results = crew.kickoff()
# print(results)


# task1 = Task(
#     description="Compile and analyze fighters' performance stats leading up to the fight",
#     agent=fighter_stats_agent,
#     data_source="ufc_stats_api.get_fighter_stats(fighter_id)"
# )

# task2 = Task(
#     description="Evaluate recent injury reports and their potential impact on the fight",
#     agent=injury_report_agent,
#     data_source="ufc_injury_api.get_recent_injuries(fighter_id)"
# )

# task3 = Task(
#     description="Review historical matchups and outcomes between the fighters",
#     agent=fight_history_agent,
#     data_source="ufc_history_api.get_fight_history(fighter1_id, fighter2_id)"
# )

# # Optional psychological analysis task
# task4 = Task(
#     description="Analyze pre-fight psychological factors from interviews",
#     agent=psychological_analysis_agent,
#     data_source="social_media_api.get_pre_fight_interviews(fighter_id)"
# )

# Instantiate the Crew and Execute
# crew = Crew(agents=[fighter_stats_agent, injury_report_agent, fight_history_agent, psychological_analysis_agent], tasks=[task1, task2, task3, task4])
# results = crew.execute()
# print(results)


In [ ]:
# Create agents
researcher = Agent(
    role='Team Research Analyst',
    goal=f'Provide up-to-date analysis of {team1}',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

In [ ]:
# UFC Agents

# Create the Fight Analyst agent
fight_analyst = Agent(
    role='Fight Analyst',
    goal='Collect and analyze detailed data on fighters and fights',
    backstory=(
        "As a seasoned analyst with a deep understanding of MMA/UFC, you specialize in breaking down fights and understanding fighter tendencies and histories."
    ),
    tools=[stats_tool]
)

# Create the Prediction Model agent
prediction_model = Agent(
    role='Prediction Model',
    goal='Predict outcomes of upcoming fights based on statistical analysis',
    backstory=(
        "Equipped with advanced predictive modeling techniques, you evaluate fighter data to forecast fight results with high accuracy."
    ),
    tools=[prediction_tool]
)

# Create the Betting Strategy agent
betting_strategy = Agent(
    role='Betting Strategy Expert',
    goal='Formulate betting strategies based on predicted outcomes',
    backstory=(
        "With a knack for numbers and risk assessment, you create informed betting strategies that maximize potential returns."
    ),
    tools=[stats_tool]
)

# Define the tasks
analysis_task = Task(
    description="Analyze fighter statistics, historical performance, and event specifics.",
    expected_output="A comprehensive dataset of analyzed fight data.",
    agent=fight_analyst
)

prediction_task = Task(
    description="Use the analyzed data to predict fight outcomes.",
    expected_output="Predicted results of upcoming fights.",
    agent=prediction_model
)

betting_task = Task(
    description="Develop betting strategies based on the predictions.",
    expected_output="Strategic betting advice for maximizing returns.",
    agent=betting_strategy
)

# Form the crew with the tasks set to be executed sequentially
crew = Crew(
    agents=[fight_analyst, prediction_model, betting_strategy],
    tasks=[analysis_task, prediction_task, betting_task]
)

# Kickoff the crew with specific inputs related to an upcoming UFC event
result = crew.kickoff(inputs={'event': 'UFC 267'})
print(result)


In [ ]:
# Sports Agents and Tasks

# Agent for analyzing team performance
team_performance_agent = Agent(
    name="Team Performance Analyst",
    role="Analyze team's past performance",
    goal="Evaluate team strengths and weaknesses based on past games"
)

# Agent for considering player injuries
player_injury_agent = Agent(
    name="Player Injury Analyst",
    role="Analyze impact of player injuries",
    goal="Assess how injuries might affect upcoming games"
)

# Agent for historical match outcomes
match_history_agent = Agent(
    name="Match History Analyst",
    role="Analyze historical match outcomes",
    goal="Understand historical performance in similar matchups"
)


# Replace 'sportsdata' with actual data source implementations
task1 = Task(
    description="Analyze team's past 5 games performance",
    agent=team_performance_agent,
    data_source="sportsdata.get_team_performance(team_id, last_n_games=5)"
)

task2 = Task(
    description="Evaluate key player injuries",
    agent=player_injury_agent,
    data_source="sportsdata.get_player_injuries(team_id)"
)

task3 = Task(
    description="Review past matchups between the two teams",
    agent=match_history_agent,
    data_source="sportsdata.get_match_history(team1_id, team2_id)"
)



crew = Crew(agents=[team_performance_agent, player_injury_agent, match_history_agent], tasks=[task1, task2, task3])
results = crew.execute()
print(results)


In [ ]:
# Random Agents

# Initialize the tool that all agents can use for searching and processing information
search_tool = SerperDevTool()

# Create the Researcher agent
researcher = Agent(
    role='Researcher',
    goal='Conduct thorough research on {topic}',
    backstory=(
        "As a tech enthusiast and skilled researcher, you have a keen eye for detail and a deep understanding of technology trends."
    ),
    tools=[search_tool]
)

# Create the Analyst agent
analyst = Agent(
    role='Analyst',
    goal='Analyze the data collected on {topic} and identify key insights',
    backstory=(
        "With a strong analytical mind, you excel at interpreting data and extracting meaningful insights from complex information."
    ),
    tools=[search_tool]
)

# Create the Writer agent
writer = Agent(
    role='Writer',
    goal='Compose a detailed, engaging article on {topic}',
    backstory=(
        "You have a talent for writing and a passion for technology, making complex topics accessible and interesting to a broad audience."
    ),
    tools=[search_tool]
)

# Define the tasks
research_task = Task(
    description="Gather comprehensive information on the latest developments in {topic}.",
    expected_output="A detailed report containing key information and latest developments.",
    agent=researcher
)

analysis_task = Task(
    description="Analyze the gathered information to identify trends, opportunities, and potential challenges.",
    expected_output="An analytical report summarizing the key findings from the research.",
    agent=analyst
)

writing_task = Task(
    description="Using the insights from the analysis, write an informative and engaging article on {topic}.",
    expected_output="A well-written, comprehensive article suitable for publication.",
    agent=writer
)

# Form the crew with the tasks set to be executed sequentially
crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task]
)

# Kickoff the crew with a specific topic
result = crew.kickoff(inputs={'topic': 'Artificial Intelligence in Healthcare'})
print(result)
